In [9]:
import pandas as pd
from datetime import datetime

# Load data
file_path = 'Historical_Data_v2.xlsx'
sheet_name = 'Historical_Data_Values'
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')

# Filter stock code and date range
df = df[
    (df['Stock_Code_Yahoo'] == 'NIFTYBANK') &
    (df['Date'] >= datetime(2023, 1, 1)) &
    (df['Date'] <= datetime(2025, 5, 16)) &
    (df['Close'] != 0)
]

# Group by year-month and get first and last close price of each month
df['YearMonth'] = df['Date'].dt.to_period('M')

monthly_data = df.groupby('YearMonth')['Close'].agg(['first', 'last']).reset_index()

# Calculate percentage move for each month
monthly_data['Percent_Move'] = ((monthly_data['last'] - monthly_data['first']) / monthly_data['first']) * 100

# Function to categorize percentage moves into buckets
def categorize_move(pct):
    if pct >= 25:
        return 'crossed_25'
    elif 20 <= pct < 25:
        return 'crossed_20_25'
    elif 15 <= pct < 20:
        return 'crossed_15_20'
    elif 10 <= pct < 15:
        return 'crossed_10_15'
    elif 5 <= pct < 10:
        return 'crossed_5_10'
    elif 0 <= pct < 5:
        return 'crossed_0_5'
    elif -5 < pct < 0:
        return 'neg_crossed_0_5'
    elif -10 < pct <= -5:
        return 'neg_crossed_5_10'
    elif -15 < pct <= -10:
        return 'neg_crossed_10_15'
    elif -20 < pct <= -15:
        return 'neg_crossed_15_20'
    elif -25 < pct <= -20:
        return 'neg_crossed_20_25'
    else:  # pct <= -25
        return 'neg_crossed_25'

monthly_data['Threshold_Category'] = monthly_data['Percent_Move'].apply(categorize_move)

# Convert YearMonth back to string (YYYY-MM)
monthly_data['YearMonth'] = monthly_data['YearMonth'].astype(str)

# Save full results to new Excel
output_file = 'NIFTYBANK_Analysis_Output.xlsx'
monthly_data.to_excel(output_file, index=False)

print(f"Analysis complete. Results saved to {output_file}")


Analysis complete. Results saved to NIFTYBANK_Analysis_Output.xlsx
